In [2]:
import os
from typing import Literal

from tavily import TavilyClient

from langchain.chat_models import init_chat_model
from deepagents import create_deep_agent

from dotenv import load_dotenv

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

load_dotenv()

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

collection_name = "transcripts"

client = QdrantClient(url="http://localhost:6333")
vector_store = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=embeddings,
)


def search_docs(question: str):
    """Search for information in the company X's earnings call data."""
    result = vector_store.similarity_search(question, k=3)
    return result


llm = init_chat_model(
    "gpt-4o-mini",
    model_provider="openai",
    temperature=0.2,
)


# Search tool to use to do research
def search_internet(
    query: str,
    max_results: int = 5,
    topic: Literal["general", "news", "finance"] = "finance",
    include_raw_content: bool = False,
):
    """Run a web search"""
    tavily_async_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
    search_docs = tavily_async_client.search(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic,
    )
    return search_docs


sub_research_prompt = """You are a dedicated researcher. Your job is to conduct research based on the users questions.

Conduct thorough research and then reply to the user with a detailed answer to their question"""

research_sub_agent = {
    "name": "research-agent",
    "description": "Used to research more in depth questions. Only give this researcher one topic at a time. Do not pass multiple sub questions to this researcher. Instead, you should break down a large topic into the necessary components, and then call multiple research agents in parallel, one for each sub question.",
    "prompt": sub_research_prompt,
}

sub_critique_prompt = """You are a dedicated editor. You are being tasked to critique a report.

You can find the report at `final_report.md`.

You can find the question/topic for this report at `question.txt`.

The user may ask for specific areas to critique the report in. Respond to the user with a detailed critique of the report. Things that could be improved.

You can use the search tool to search for information, if that will help you critique the report

Do not write to the `final_report.md` yourself.

Things to check:
- Check that each section is appropriately named
- Check that the report is written as you would find in an essay or a textbook - it should be text heavy, do not let it just be a list of bullet points!
- Check that the report is comprehensive. If any paragraphs or sections are short, or missing important details, point it out.
- Check that the article covers key areas of the industry, ensures overall understanding, and does not omit important parts.
- Check that the article deeply analyzes causes, impacts, and trends, providing valuable insights
- Check that the article closely follows the research topic and directly answers questions
- Check that the article has a clear structure, fluent language, and is easy to understand.
"""

critique_sub_agent = {
    "name": "critique-agent",
    "description": "Used to critique the final report. Give this agent some infomration about how you want it to critique the report.",
    "prompt": sub_critique_prompt,
}


# Prompt prefix to steer the agent to be an expert researcher
research_instructions = """You are an expert researcher. Your job is to conduct thorough research, and then write a polished report.

The first thing you should do is to write the original user question to `question.txt` so you have a record of it.

Use the research-agent to conduct deep research. It will respond to your questions/topics with a detailed answer.

When you think you enough information to write a final report, write it to `final_report.md`

You can call the critique-agent to get a critique of the final report. After that (if needed) you can do more research and edit the `final_report.md`
You can do this however many times you want until are you satisfied with the result.

Only edit the file once at a time (if you call this tool in parallel, there may be conflicts).

Here are instructions for writing the final report:

<report_instructions>

CRITICAL: Make sure the answer is written in the same language as the human messages!
For example, if the user's messages are in English, then MAKE SURE you write your response in English. If the user's messages are in Chinese, then MAKE SURE you write your entire response in Chinese.
This is critical. The user will only understand the answer if it is written in the same language as their input message.

Please create a detailed answer to the overall research brief that:
1. Is well-organized with proper headings (# for title, ## for sections, ### for subsections)
2. Includes specific facts and insights from the research
3. References relevant sources using [Title](URL) format
4. Provides a balanced, thorough analysis. Be as comprehensive as possible, and include all information that is relevant to the overall research question. People are using you for deep research and will expect detailed, comprehensive answers.
5. Includes a "Sources" section at the end with all referenced links

You can structure your report in a number of different ways. Here are some examples:

To answer a question that asks you to compare two things, you might structure your report like this:
1/ intro
2/ overview of topic A
3/ overview of topic B
4/ comparison between A and B
5/ conclusion

To answer a question that asks you to return a list of things, you might only need a single section which is the entire list.
1/ list of things or table of things
Or, you could choose to make each item in the list a separate section in the report. When asked for lists, you don't need an introduction or conclusion.
1/ item 1
2/ item 2
3/ item 3

To answer a question that asks you to summarize a topic, give a report, or give an overview, you might structure your report like this:
1/ overview of topic
2/ concept 1
3/ concept 2
4/ concept 3
5/ conclusion

If you think you can answer the question with a single section, you can do that too!
1/ answer

REMEMBER: Section is a VERY fluid and loose concept. You can structure your report however you think is best, including in ways that are not listed above!
Make sure that your sections are cohesive, and make sense for the reader.

For each section of the report, do the following:
- Use simple, clear language
- Use ## for section title (Markdown format) for each section of the report
- Do NOT ever refer to yourself as the writer of the report. This should be a professional report without any self-referential language. 
- Do not say what you are doing in the report. Just write the report without any commentary from yourself.
- Each section should be as long as necessary to deeply answer the question with the information you have gathered. It is expected that sections will be fairly long and verbose. You are writing a deep research report, and users will expect a thorough answer.
- Use bullet points to list out information when appropriate, but by default, write in paragraph form.

REMEMBER:
The brief and research may be in English, but you need to translate this information to the right language when writing the final answer.
Make sure the final answer report is in the SAME language as the human messages in the message history.

Format the report in clear markdown with proper structure and include source references where appropriate.

<Citation Rules>
- Assign each unique URL a single citation number in your text
- End with ### Sources that lists each source with corresponding numbers
- IMPORTANT: Number sources sequentially without gaps (1,2,3,4...) in the final list regardless of which sources you choose
- Each source should be a separate line item in a list, so that in markdown it is rendered as a list.
- Example format:
  [1] Source Title: URL
  [2] Source Title: URL
- Citations are extremely important. Make sure to include these, and pay a lot of attention to getting these right. Users will often use these citations to look into more information.
</Citation Rules>
</report_instructions>

You have access to a few tools.

## `internet_search`

Use this to run an internet search for a given query. You can specify the number of results, the topic, and whether raw content should be included.
"""

In [ ]:
# agent.invoke({"messages": [{"role": "user", "content": "For Tata Consultancy Services (TCS) analyze the financial reports and transcripts for the past quarters and provide a qualitative forecast for the upcoming quarter. Your forecast must identify key financial trends (e.g., revenue growth, margin pressure), summarize management's stated outlook, and highlight any significant risks or opportunities mentioned."}]})

{'messages': [HumanMessage(content="For Tata Consultancy Services (TCS) analyze the financial reports and transcripts for the past quarters and provide a qualitative forecast for the upcoming quarter. Your forecast must identify key financial trends (e.g., revenue growth, margin pressure), summarize management's stated outlook, and highlight any significant risks or opportunities mentioned.", additional_kwargs={}, response_metadata={}, id='7eae0d51-9fd7-44a0-b14c-f46f60fe57ab'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mPmNTPuiWGGg14TlJ4hHZaJa', 'function': {'arguments': '{"file_path":"question.txt","content":"For Tata Consultancy Services (TCS) analyze the financial reports and transcripts for the past quarters and provide a qualitative forecast for the upcoming quarter. Your forecast must identify key financial trends (e.g., revenue growth, margin pressure), summarize management\'s stated outlook, and highlight any significant risks or opportunities ment

In [ ]:
finance_instructions = """You are a financial research specialist. Your job is to conduct deep, qualitative research and produce a professional-grade report focused on financial performance, trends, and forward-looking insights.

Start by saving the original user request to `question.txt` for recordkeeping.

Use the `research-agent` to gather information. This may include earnings call transcripts, quarterly/annual financial reports, investor presentations, news coverage, and analyst commentary.

Once you have gathered sufficient insights, write your report to `final_report.md`.

You may call the `critique-agent` for feedback on your draft. Based on the critique, perform additional research or revise the report as needed. Repeat this loop until you are satisfied with the final product.

Only write to one file at a time to avoid conflicts.

---

## Instructions for Writing the Final Report

<report_instructions>

CRITICAL: The report must be in the same language as the user's message. If the user writes in English, respond in English. If they write in Chinese, respond in Chinese. Do not mix languages.

### Focus and Purpose

Your report must deliver a **qualitative financial forecast** grounded in evidence. Specifically, your analysis should:

1. Interpret recent quarterly earnings (typically the past three quarters)
2. Extract and synthesize key financial metrics, such as:
   - Total Revenue
   - Net Income
   - Operating Margin
   - Free Cash Flow
   - Segment performance (if applicable)
3. Identify financial trends (e.g., revenue acceleration, margin compression, inventory buildup)
4. Extract forward-looking statements from company management, including:
   - Guidance revisions
   - Business outlook
   - Strategic priorities
   - Commentary on macroeconomic headwinds/tailwinds
5. Analyze management tone/sentiment and recurring themes across transcripts
6. Highlight material risks and potential opportunities mentioned in earnings calls or filings
7. Conclude with a **qualitative forecast** for the next quarter or upcoming fiscal period based on the above

### Report Structure Guidelines

Adapt your structure to the task. Here are some common patterns:

- **Quarterly Analysis & Forecast**:  
  1/ Executive Summary  
  2/ Key Financial Highlights (past 3 quarters)  
  3/ Thematic Analysis (sentiment, risks, opportunities)  
  4/ Management Guidance & Forward-Looking Statements  
  5/ Qualitative Forecast  
  6/ Conclusion  
  7/ Sources  

- **Comparison Across Peers or Time Periods**:  
  1/ Introduction  
  2/ Company A performance  
  3/ Company B performance  
  4/ Comparative Analysis  
  5/ Conclusion

- **Standalone Trend Analysis**:  
  1/ Overview  
  2/ Trend or Theme 1  
  3/ Trend or Theme 2  
  4/ Outlook  
  5/ Sources

Customize structure as needed. Cohesion, flow, and clarity are key.

---

## Writing Rules

- Use `##` for section titles (Markdown format)
- Use `###` for subsections if needed
- Avoid self-referential language—do not mention yourself or the process
- Be comprehensive, even if verbose—depth is expected
- Use bullet points only when clearly helpful; prefer paragraphs otherwise
- Translate content as needed to match the language of the user’s original message
- Write in a formal, analytic tone suitable for financial professionals

---

You have access to the following tools:

## `search_documents
## `search_internet`

Use these to find relevant financial data, earnings call transcripts, analyst reports, and other information needed to answer the user's request."""
# Create the agent
agent = create_deep_agent(
    [search_docs, search_internet],
    finance_instructions,
    subagents=[critique_sub_agent, research_sub_agent],
    model=llm,
).with_config({"recursion_limit": 20})

In [4]:
response = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "For Tata Consultancy Services (TCS) analyze the financial reports and transcripts for this year and provide a qualitative forecast for the upcoming quarter. Your forecast must identify key financial trends (e.g., revenue growth, margin pressure), summarize management's stated outlook, and highlight any significant risks or opportunities mentioned.",
            }
        ]
    }
)